# AUDIO WRANGLING 🔊

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from pydub.utils import make_chunks
from pydub import AudioSegment
from scipy.fftpack import fft
from pydub.utils import make_chunks
import numpy as np
import pandas as pd
import re

----

In [2]:
path = "../input"
for familia in os.listdir(path):
    familia = os.path.join(path, familia)
    print(familia)

../input/.DS_Store
../input/Bufonidae
../input/Arthroleptidae
../input/Hyperollidae
../input/Leptodactylidae
../input/Dendrobatidae


In [3]:
def audioSegmentData(direc, chunk_length_ms):
    fam = []
    path = f"../input/{direc}"
    for audio in os.listdir(path):
        if audio.endswith('.mp3'):
            croak = AudioSegment.from_mp3(path+"/"+audio)
            croak = croak.set_frame_rate(16000)
            
            croak_chunks = make_chunks(croak, chunk_length_ms)
            fam.append(croak_chunks)
    return fam

---

## Segmenting Recordings

In [4]:
buf = audioSegmentData("Bufonidae", 5000)
buf = [x for e in buf for x in e]

In [5]:
den = audioSegmentData("Dendrobatidae", 5000)
den = [x for e in den for x in e]

In [6]:
hyp = audioSegmentData("Hyperollidae", 5000)
hyp = [x for e in hyp for x in e]

In [7]:
lep = audioSegmentData("Leptodactylidae", 5000)
lep = [x for e in lep for x in e]

-------

------

## Extracting Fourier Features


In [ ]:
# au.audioFeaturesFourie(buf)

In [8]:
def audioFeaturesFourier(array):
    for e in array:
        array = e.get_array_of_samples()
        abs_four = np.abs(fft(array))
    return abs_four

In [9]:
bufFF = audioFeaturesFourier(buf)
bufFF

array([18001.        ,  2077.43801242,  4324.44767849, ...,
        7462.54575626,  4324.44767849,  2077.43801242])

In [10]:
denFF = audioFeaturesFourier(den)
denFF

array([5302.        , 6228.34096784, 6393.50316783, ..., 5637.85263854,
       6393.50316783, 6228.34096784])

In [11]:
hypFF = audioFeaturesFourier(hyp)
hypFF

array([87513.        ,  5053.06608667, 60770.5592036 , ...,
       95014.37986937, 60770.5592036 ,  5053.06608667])

In [12]:
lepFF = audioFeaturesFourier(lep)
lepFF

array([125549.        , 178438.35068799, 202574.36759407, ...,
       129512.73696723, 202574.36759407, 178438.35068799])

-----

----

## Creating *Familia* Dataframes

In [13]:
buf_or = {"F. Transform": [a for a in bufFF], "Tag": 1}
den_or = {"F. Transform": [a for a in denFF], "Tag": 2}
hyp_or = {"F. Transform": [a for a in hypFF], "Tag": 3}
lep_or = {"F. Transform": [a for a in lepFF], "Tag": 4}

In [14]:
w = pd.DataFrame(buf_or)
x = pd.DataFrame(den_or)
y = pd.DataFrame(hyp_or)
z = pd.DataFrame(lep_or)

totus = pd.concat([w, x, y, z]).reset_index(drop=True)
totus

,F. Transform,Tag
0,18001.000000,1
1,2077.438012,1
2,4324.447678,1
3,7462.545756,1
4,8746.229687,1
...,...,...
364827,190248.712618,4
364828,118798.087221,4
364829,129512.736967,4
364830,202574.367594,4


## Classifying with get_dummies

In [19]:
dum_totus = pd.get_dummies(totus, prefix=['Tag'], columns=['Tag'])
dum_totus

,F. Transform,Tag_1,Tag_2,Tag_3,Tag_4
0,18001.000000,1,0,0,0
1,2077.438012,1,0,0,0
2,4324.447678,1,0,0,0
3,7462.545756,1,0,0,0
4,8746.229687,1,0,0,0
...,...,...,...,...,...
364827,190248.712618,0,0,0,1
364828,118798.087221,0,0,0,1
364829,129512.736967,0,0,0,1
364830,202574.367594,0,0,0,1


----

---

## Exporting the df to csv

In [29]:
# totus.to_csv('../output/fam_df.csv')

In [21]:
# dum_totus.to_csv('../output/fam_dum_df.csv')